In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pymongo
from pymongo import MongoClient
import re

In [2]:
start = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
resp = requests.get(start, headers=headers)


In [4]:
resp

<Response [200]>

In [5]:
soup = BeautifulSoup(resp.content, "html.parser")

In [6]:
soup

<!DOCTYPE html>

<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<!-- Always force latest IE rendering engine or request Chrome Frame -->
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"5e33925808","applicationID":"59562082","transactionName":"JVcPR0MLWApSRU1eAQVVEhxSC1oSUlkWbBMHXwRAHhdcCUA=","queueTime":0,"applicationTime":174,"agent":""}</script>
<script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"VQcPUlZTDxAFXVRUBQEPVA=="};window.NREUM||(NREUM={}),__nr_require=function(t,n,e){function r(e){if(!n[e]){var o=n[e]={exports:{}};t[e][0].call(o.exports,function(n){var o=t[e][1][n];return r(o||n)},o,o.exports)}return n[e].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<e.length;o++)r(e[o

In [7]:
rows = soup.find_all("div", {"class": "content_title"})

In [8]:
rows[0]

<div class="content_title">
<a href="/news/8413/nasas-opportunity-rover-mission-on-mars-comes-to-end/">
NASA's Opportunity Rover Mission on Mars Comes to End
</a>
</div>

In [9]:
url = "https://mars.nasa.gov/api/v1/news_items/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&year=2019%3Apublish_date&category=19%2C165%2C184%2C204&blank_scope=Latest"

In [10]:
resp = requests.get(url)

In [11]:
resp

<Response [200]>

In [12]:
data = resp.json()

In [13]:
data

{'items': [{'id': 8416,
   'title': 'After a Reset, Curiosity Is Operating Normally',
   'target': '_self',
   'main_image_alt': 'NASA\'s Curiosity Mars took this image with its Mastcam on Feb. 10, 2019 (Sol 2316). The rover is currently exploring a region of Mount Sharp nicknamed "Glen Torridon" that has lots of clay minerals.',
   'updated_at': '2019-02-28T18:06:07.682Z',
   'meta_author': '',
   'byline': '',
   'meta_keywords': '',
   'item_name': '',
   'url': '/news/8416/after-a-reset-curiosity-is-operating-normally/',
   'missions': [[175, 'mep', 'Mars Exploration Program'],
    [176, 'msl', 'Mars Science Laboratory']],
   'news_type': [[204, 'status_reports', 'Status Reports']],
   'description': 'Curiosity has returned to science operations and is once again exploring the clay unit. ',
   'date': 'February 28, 2019',
   'body': '<p><b>Updated Feb. 28, 2019, 8:30 a.m. PST (11:30 a.m. EST)</b></p>\n\n<p>Curiosity has returned to science operations and is once again exploring the

In [14]:
data.keys()

dict_keys(['items', 'more', 'total', 'page', 'per_page'])

In [15]:
df = pd.DataFrame.from_records(data)

In [16]:
df.head()

,items,more,page,per_page,total
0,"{'id': 8416, 'title': 'After a Reset, Curiosit...",False,0,40,15
1,"{'id': 8417, 'title': 'InSight Lands Praise an...",False,0,40,15
2,"{'id': 8415, 'title': 'InSight Is the Newest M...",False,0,40,15
3,"{'id': 8414, 'title': 'Six Things to Know Abou...",False,0,40,15
4,"{'id': 8413, 'title': 'NASA's Opportunity Rove...",False,0,40,15


In [17]:
df['title'] = df['items'].map(lambda x : x.get('title'))

In [18]:
df.head(1)

,items,more,page,per_page,total,title
0,"{'id': 8416, 'title': 'After a Reset, Curiosit...",False,0,40,15,"After a Reset, Curiosity Is Operating Normally"


In [19]:
df["news_descri"]= df['items'].map(lambda x : x.get('description'))

In [20]:
df.head(1)

,items,more,page,per_page,total,title,news_descri
0,"{'id': 8416, 'title': 'After a Reset, Curiosit...",False,0,40,15,"After a Reset, Curiosity Is Operating Normally",Curiosity has returned to science operations a...


In [21]:
pic_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
pic_data = requests.get(pic_url)

In [22]:
pic_data

<Response [200]>

In [23]:
soup = BeautifulSoup(pic_data.content, "html.parser")

In [24]:
soup

<!DOCTYPE html>

<!--[if IE 9]> <html class="no-js ie ie9" lang="en"> <![endif]-->
<!--[if IE 8]> <html class="no-js ie ie8" lang="en"> <![endif]-->
<html>
<!-- START HEADER: "DEFAULT" -->
<head>
<meta charset="utf-8"/>
<!-- Always force latest IE rendering engine or request Chrome Frame -->
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/> <title>Space Images</title>
<link href="/assets/stylesheets/manifest.css" media="all" rel="stylesheet" type="text/css"/>
<link href="/assets/stylesheets/print.css" media="print" rel="stylesheet" type="text/css"/>
<script src="/assets/javascripts/public_manifest.js" type="text/javascript"></script>
<script src="/assets/javascripts/vendor/jquery.fancybox.js" type="text/javascript"></script>
<script src="/assets/javascripts/vendor/jquery.fancybox-thumbs.js" type="text/javascript"></script>
</head>
<body class="dark_background logge

In [25]:
pic_link_temp = soup.find('a', {"class":"fancybox"}).attrs.get("data-fancybox-href")

In [26]:
pic_link_temp

'/spaceimages/images/mediumsize/PIA14884_ip.jpg'

In [27]:
pic_link = "https://www.jpl.nasa.gov" + pic_link_temp

In [28]:
pic_link

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA14884_ip.jpg'

In [29]:
weather_url = "https://twitter.com/MarsWxReport/status/1098709468945281025"

In [30]:
weather_data = requests.get(weather_url)

In [31]:
weather_data

<Response [200]>

In [32]:
soup_weather = BeautifulSoup(weather_data.content, "html.parser")

In [33]:
soup_weather

<!DOCTYPE html>

<html data-scribe-reduced-action-queue="true" lang="en">
<head>
<meta charset="utf-8"/>
<script nonce="OXwwd7iFkM7smtTWL0iHhw==">
        !function(){window.initErrorstack||(window.initErrorstack=[]),window.onerror=function(r,i,n,o,t){r.indexOf("Script error.")>-1||window.initErrorstack.push({errorMsg:r,url:i,lineNumber:n,column:o,errorObj:t})}}();
      </script>
<script id="bouncer_terminate_iframe" nonce="OXwwd7iFkM7smtTWL0iHhw==">
    if (window.top != window) {
  window.top.postMessage({'bouncer': true, 'event': 'complete'}, '*');
}
  </script>
<script id="swift_action_queue" nonce="OXwwd7iFkM7smtTWL0iHhw==">
    !function(){function e(e){if(e||(e=window.event),!e)return!1;if(e.timestamp=(new Date).getTime(),!e.target&&e.srcElement&&(e.target=e.srcElement),document.documentElement.getAttribute("data-scribe-reduced-action-queue"))for(var t=e.target;t&&t!=document.body;){if("A"==t.tagName)return;t=t.parentNode}return i("all",o(e)),a(e)?(document.addEventListener||(e

In [34]:
weather_description= soup_weather.find('p', {"class":"TweetTextSize"}).get_text()

In [35]:
weather_description

'InSight sol 84 (2019-02-20) low -95.1ºC (-139.2ºF) high -13.2ºC (8.3ºF)\nwinds from the SW at 4.1 m/s (9.3 mph) gusting to 10.8 m/s (24.2 mph)pic.twitter.com/WlR4gr8gpC'

In [36]:
df_table = pd.read_html("https://space-facts.com/mars/")

In [37]:
df_table

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [38]:
homepage = requests. get("https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars")

In [39]:
homepage

<Response [200]>

In [40]:
soup = BeautifulSoup(homepage.content, "html.parser")

In [41]:
links = soup.find_all('a', {'class':'itemLink'})

In [42]:
links

[<a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/dfaf3849e74bf973b59eb50dab52b583_cerberus_enhanced.tif_thumb.png"/><div class="description"><h3>Cerberus Hemisphere Enhanced</h3></div></a>,
 <a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/7677c0a006b83871b5a2f66985ab5857_schiaparelli_enhanced.tif_thumb.png"/><div class="description"><h3>Schiaparelli Hemisphere Enhanced</h3></div></a>,
 <a class="itemLink product-item" href="/search/map/Mars/Viking/syrtis_major_enhanced"><img alt="Syrtis Major Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/aae41197e40d6d4f3ea557f8cfe51d15_syrtis_major_enhanced.tif_thumb.png"/><div class="description"><h3>Syrtis Major Hemisphere Enhanced</h3></div></a>,
 <a class="itemLink product-item" href="/search

In [43]:
links = ["https://astrogeology.usgs.gov/" + x.attrs.get('href') for x in links]

In [44]:
links

['https://astrogeology.usgs.gov//search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov//search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov//search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov//search/map/Mars/Viking/valles_marineris_enhanced']

In [45]:
ls

Images/                       data.txt
Jeff stuff from Feb 28.ipynb  mission_to_mars.ipynb
README.md                     scrape_mars.py
__pycache__/                  templates/
app.py                        web_pages/


In [46]:
%mkdir web_pages

mkdir: web_pages: File exists


In [47]:
ls


Images/                       data.txt
Jeff stuff from Feb 28.ipynb  mission_to_mars.ipynb
README.md                     scrape_mars.py
__pycache__/                  templates/
app.py                        web_pages/


In [48]:
links[1].split("/")[-1]

'schiaparelli_enhanced'

In [49]:
import os

def save_page_from_web(url):
    resp = requests.get(url)
    save_name = url.split("/")[-1] + ".html"
    save_path = os.path.join("web_pages/",save_name )
    with open(save_path, "w") as f:
        f.write(resp.text)
    

In [50]:
for link in links:
    save_page_from_web(link)

In [51]:
import glob


In [52]:
html = glob.glob('web_pages/*.html')
html

['web_pages/valles_marineris_enhanced.html',
 'web_pages/cerberus_enhanced.html',
 'web_pages/syrtis_major_enhanced.html',
 'web_pages/schiaparelli_enhanced.html']

In [53]:
results = []

for local_file in html:
    with open(local_file, 'r') as f:
        soup = BeautifulSoup(f, "html.parser")
        full_image_link = soup.find('div', {'class': 'downloads'}).find('ul').find_all('li')[1].find('a').attrs.get('href')
        image_name = soup.find('section', {'class': 'block'}).find('h2', {'class':'title'}).get_text()
        results.append({
            "image_link": full_image_link,
            "image_name": image_name
        })

In [54]:
results

[{'image_link': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif',
  'image_name': 'Valles Marineris Hemisphere Enhanced'},
 {'image_link': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif',
  'image_name': 'Cerberus Hemisphere Enhanced'},
 {'image_link': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif',
  'image_name': 'Syrtis Major Hemisphere Enhanced'},
 {'image_link': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif',
  'image_name': 'Schiaparelli Hemisphere Enhanced'}]

In [55]:
mars_collections={}
mars_collections['hemisphere_image'] = results

In [56]:
mars_collections['hemisphere_image'][0]['image_name']

'Valles Marineris Hemisphere Enhanced'

In [ ]:
# the following coding is for scrape.just for notes

In [ ]:
full_image_link


In [ ]:
image_name = soup.find('section', {'class': 'block'}).find('h2', {'class':'title'}).get_text()

In [ ]:
image_name

In [ ]:
full_image_link = soup.find('div', {'class': 'downloads'}).find("img",{"class":"thumb"}).attrs.get("herf")

In [ ]:
full_image_link

In [62]:
full_image_link = "https://astrogeology.usgs.gov"+soup.find('img', {'class': 'wide-image'}).attrs.get("src")

In [63]:
full_image_link

'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'

In [67]:
full_image_name = soup.find('h2', {'class': 'title'}).get_text()

In [68]:
full_image_name

'Schiaparelli Hemisphere Enhanced'